# ∞-Sector Convergence Demo

This notebook illustrates a toy version of the VID ∞-sector convergence using a finite-dimensional operator.

We use the pseudoinverse of the DLSFH Laplacian, `Delta20_pinv.npy`, to construct a bounded operator
$K$ with spectral radius strictly less than 1, and then study the convergence of the series
$$ S_N = \sum_{n=0}^N K^n. $$

In an idealized ∞-sector picture, one considers an infinite tower of operators; here we approximate the
∞-sector limit with a large cutoff $N_{\max}$ and track two error metrics
$$ \mathrm{err}_2(N) = \lVert S_N - S_{N_{\max}} \rVert_2, \qquad
   \mathrm{err}_F(N) = \lVert S_N - S_{N_{\max}} \rVert_F, $$
as $N$ increases.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Paths can be adjusted depending on where you run the notebook
L_pinv_path = "../data/Delta20_pinv.npy"  # or "Delta20_pinv.npy" if run from repo root

L_pinv = np.load(L_pinv_path)
print("L^+ shape:", L_pinv.shape)

# Compute operator norm (2-norm) of L^+
op_norm = np.linalg.norm(L_pinv, ord=2)
print("Operator norm ||L^+||_2 =", op_norm)

# Build a bounded operator K with spectral radius < 1
# Here we use K = alpha * (L^+ / ||L^+||_2) with alpha in (0,1)
alpha = 0.7
K = alpha * (L_pinv / op_norm)
print("Approximate operator norm ||K||_2 <=", alpha)


## Constructing the partial sums $S_N$

We define
$$ S_N = I + K + K^2 + \cdots + K^N. $$

For a fixed cutoff $N_{\max}$, we treat $S_{N_{\max}}$ as an approximation to the ∞-sector limit and
measure the convergence of $S_N$ via both the operator 2-norm and Frobenius norm.

In [ ]:
N_max = 50  # proxy for "infinity" in this finite example

dim = K.shape[0]
I = np.eye(dim)

# First, build S_{N_max}
S_Nmax = np.zeros_like(K)
current = I.copy()
for n in range(N_max + 1):
    S_Nmax += current
    current = current @ K

print("Built S_Nmax with N_max =", N_max)

# Now build all partial sums S_N and track errors relative to S_Nmax
errors_2 = []
errors_fro = []
Ns = list(range(N_max + 1))

S_N = np.zeros_like(K)
current = I.copy()
for N in Ns:
    # add the next term
    if N == 0:
        S_N = current.copy()
    else:
        S_N += current
    # compute errors vs S_Nmax
    diff = S_N - S_Nmax
    err2 = np.linalg.norm(diff, ord=2)
    errF = np.linalg.norm(diff, ord="fro")
    errors_2.append(err2)
    errors_fro.append(errF)
    # prepare next power
    current = current @ K

errors_2 = np.array(errors_2)
errors_fro = np.array(errors_fro)
print("Initial errors (N=0): ||.||_2 =", errors_2[0], ", ||.||_F =", errors_fro[0])
print("Final errors (N=N_max): ||.||_2 =", errors_2[-1], ", ||.||_F =", errors_fro[-1])


## Convergence plots

We now plot both error metrics
$$ \mathrm{err}_2(N), \quad \mathrm{err}_F(N) $$
as functions of $N$. A roughly exponential decay in $N$ is a numerical indication of ∞-sector convergence
in this toy model.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(Ns, errors_2, marker="o", label=r"$||S_N - S_{N_{max}}||_2$")
ax.plot(Ns, errors_fro, marker="s", label=r"$||S_N - S_{N_{max}}||_F$")
ax.set_yscale("log")
ax.set_xlabel("N (partial sum index)")
ax.set_ylabel("Error (log scale)")
ax.set_title(r"∞-sector convergence: partial sums of $K^n$")
ax.grid(True, which="both", ls=":", alpha=0.6)
ax.legend()

plt.tight_layout()
plt.show()

### Summary

- We built a bounded operator $K$ from the Laplacian pseudoinverse.
- We formed finite partial sums $S_N = \sum_{n=0}^N K^n$.
- For a fixed $N_{\max}$, we observed monotone decay of both $\mathrm{err}_2(N)$ and $\mathrm{err}_F(N)$.

This toy model numerically demonstrates the type of convergence behavior expected in the VID ∞-sector, where
an infinite tower of discrete geometric operators defines a well-behaved nonperturbative object.
